In [1]:
import os
import sys
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
os.chdir('../..')
os.getcwd()

'c:\\Users\\user\\Downloads\\ten_academy\\week1\\Week1-10Academy-User-Analysis'

In [3]:
from src.plots import Plot

In [11]:
df = pd.read_csv('data/tel_dump.csv')

In [5]:
plot = Plot()

In [12]:
df.columns

Index(['Bearer Id', 'Start', 'Start ms', 'End', 'End ms', 'Dur. (ms)', 'IMSI',
       'MSISDN/Number', 'IMEI', 'Last Location Name', 'Avg RTT DL (ms)',
       'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
       'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)',
       'Activity Duration UL (ms)', 'Dur. (ms).1', 'Handset Manufacturer',
       'Handset Type', 'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

In [14]:
df.isna().sum()

Bearer Id                                      991
Start                                            1
Start ms                                         1
End                                              1
End ms                                           1
Dur. (ms)                                        1
IMSI                                           570
MSISDN/Number                                 1066
IMEI                                           572
Last Location Name                            1153
Avg RTT DL (ms)                              27829
Avg RTT UL (ms)                              27812
Avg Bearer TP DL (kbps)                          1
Avg Bearer TP UL (kbps)                          1
TCP DL Retrans. Vol (Bytes)                  88146
TCP UL Retrans. Vol (Bytes)                  96649
DL TP < 50 Kbps (%)                            754
50 Kbps < DL TP < 250 Kbps (%)                 754
250 Kbps < DL TP < 1 Mbps (%)                  754
DL TP > 1 Mbps (%)             

In [26]:
df = df.query("`Handset Manufacturer` != 'undefined' and `Handset Type` != 'undefined'")[df.columns]

In [27]:
# Data Cleaning: Handling missing values for relevant columns

# Fill missing numerical values with the mean of the respective column
df['TCP DL Retrans. Vol (Bytes)'] = df['TCP DL Retrans. Vol (Bytes)'].fillna(df['TCP DL Retrans. Vol (Bytes)'].mean())
df['TCP UL Retrans. Vol (Bytes)'] = df['TCP UL Retrans. Vol (Bytes)'].fillna(df['TCP UL Retrans. Vol (Bytes)'].mean())
df['Avg RTT DL (ms)'] = df['Avg RTT DL (ms)'].fillna(df['Avg RTT DL (ms)'].mean())
df['Avg RTT UL (ms)'] = df['Avg RTT UL (ms)'].fillna(df['Avg RTT UL (ms)'].mean())


# Fill missing categorical values with the mode
df['Handset Type'] = df['Handset Type'].fillna(df['Handset Type'].mode()[0])
df['MSISDN/Number'] = df['MSISDN/Number'].fillna(df['MSISDN/Number'].mode()[0])

# Confirm that missing values have been addressed in these columns
cleaned_data_info = df[['TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Handset Type', 'MSISDN/Number']].isnull().sum()

cleaned_data_info  # Output to confirm the missing values have been handled


TCP DL Retrans. Vol (Bytes)    0
TCP UL Retrans. Vol (Bytes)    0
Avg RTT DL (ms)                0
Avg RTT UL (ms)                0
Handset Type                   0
MSISDN/Number                  0
dtype: int64

In [29]:
# Task 4.1: Aggregate information per customer
# Group by customer (using 'IMSI') and calculate the required metrics

# Calculate average TCP retransmission for each customer
# Sum DL and UL retransmission values and divide by the count for averaging
avg_tcp_retransmission = df.groupby('MSISDN/Number')[['TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)']].sum().sum(axis=1) / df.groupby('MSISDN/Number').size()

# Calculate average RTT for each customer (both downlink and uplink)
avg_rtt = df.groupby('MSISDN/Number')[['Avg RTT DL (ms)', 'Avg RTT UL (ms)']].mean().mean(axis=1)

# Calculate average throughput for each customer (both downlink and uplink)
avg_throughput = df.groupby('MSISDN/Number')[['Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)']].mean().mean(axis=1)

# Identify the most frequent handset type for each customer
most_frequent_handset_type = df.groupby('MSISDN/Number')['Handset Type'].agg(lambda x: x.mode()[0])

# Combine the calculated metrics into a single dataframe
customer_experience_aggregate = pd.DataFrame({
    'MSISDN/Number': avg_tcp_retransmission.index,
    'Average TCP Retransmission': avg_tcp_retransmission.values,
    'Average RTT': avg_rtt.values,
    'Most Frequent Handset Type': most_frequent_handset_type.values,
    'Average Throughput (kbps)': avg_throughput.values
})

customer_experience_aggregate.head()  # Display the first few rows of the aggregated information per customer


,MSISDN/Number,Average TCP Retransmission,Average RTT,Most Frequent Handset Type,Average Throughput (kbps)
0,3.360100e+10,2.156957e+07,23.000000,Huawei P20 Lite Huawei Nova 3E,38.0
1,3.360100e+10,2.156957e+07,15.500000,Apple iPhone 7 (A1778),49.5
2,3.360101e+10,7.607247e+05,42.000000,Apple iPhone 5S (A1457),124.0
3,3.360101e+10,1.547020e+07,29.750000,Apple iPhone Se (A1723),14211.0
4,3.360101e+10,1.116600e+07,37.864647,Samsung Galaxy A8 (2018),1977.0


In [30]:
# Task 4.2: Compute and list 10 of the top, bottom, and most frequent values for TCP retransmission, RTT, and throughput

# Define a function to get top, bottom, and most frequent values
def get_top_bottom_frequent(series, n=10):
    top_values = series.nlargest(n)  # Top values
    bottom_values = series.nsmallest(n)  # Bottom values
    most_frequent_values = series.value_counts().nlargest(n)  # Most frequent values
    return top_values, bottom_values, most_frequent_values

# Top, bottom, and most frequent TCP retransmission values
tcp_dl = df['TCP DL Retrans. Vol (Bytes)']
tcp_ul = df['TCP UL Retrans. Vol (Bytes)']
tcp_total = tcp_dl + tcp_ul  # Total TCP retransmission

top_tcp, bottom_tcp, most_frequent_tcp = get_top_bottom_frequent(tcp_total)

# Top, bottom, and most frequent RTT values
rtt_dl = df['Avg RTT DL (ms)']
rtt_ul = df['Avg RTT UL (ms)']
rtt_total = (rtt_dl + rtt_ul) / 2  # Average RTT

top_rtt, bottom_rtt, most_frequent_rtt = get_top_bottom_frequent(rtt_total)

# Top, bottom, and most frequent throughput values
throughput_dl = df['Avg Bearer TP DL (kbps)']
throughput_ul = df['Avg Bearer TP UL (kbps)']
throughput_total = (throughput_dl + throughput_ul) / 2  # Average throughput

top_throughput, bottom_throughput, most_frequent_throughput = get_top_bottom_frequent(throughput_total)

# Create a dictionary with the results for easy output
tcp_results = {
    "Top 10 TCP values": top_tcp,
    "Bottom 10 TCP values": bottom_tcp,
    "Most frequent 10 TCP values": most_frequent_tcp
}

rtt_results = {
    "Top 10 RTT values": top_rtt,
    "Bottom 10 RTT values": bottom_rtt,
    "Most frequent 10 RTT values": most_frequent_rtt
}

throughput_results = {
    "Top 10 Throughput values": top_throughput,
    "Bottom 10 Throughput values": bottom_throughput,
    "Most frequent 10 Throughput values": most_frequent_throughput
}

tcp_results, rtt_results, throughput_results  # Display the results for each category


({'Top 10 TCP values': 34645     4.344116e+09
  140813    4.301477e+09
  77979     4.294432e+09
  135678    4.291465e+09
  3782      4.288121e+09
  119684    4.275492e+09
  39637     4.268647e+09
  59016     4.261616e+09
  76990     4.260174e+09
  41209     4.254660e+09
  dtype: float64,
  'Bottom 10 TCP values': 143594     86.0
  111113     97.0
  18432     106.0
  13157     108.0
  97070     113.0
  8816      128.0
  99959     129.0
  95966     134.0
  12866     143.0
  50022     173.0
  dtype: float64,
  'Most frequent 10 TCP values': 2.156957e+07    78955
  2.081121e+07      634
  7.609887e+05      244
  2.081124e+07      233
  7.596967e+05      136
  7.597507e+05      132
  2.081123e+07      131
  7.623187e+05      121
  7.609767e+05      100
  2.081257e+07       92
  Name: count, dtype: int64},
 {'Top 10 RTT values': 30166     48462.0
  29927     32320.5
  17910     27906.0
  5989      27424.0
  22851     13639.0
  23455     13150.0
  1373      12961.0
  81274     12857.5
  97321